In [2]:
!pip install numpy
!pip install scipy
!pip install Pillow
!pip install cython
!pip install matplotlib
!pip install scikit-image
!pip install tensorflow==1.14.0
!pip install keras==2.3.1
!pip install opencv-python
!pip install h5py
!pip install imgaug
!pip install IPython

In [3]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
from PIL import Image

input_img = "/content/drive/My Drive/Colab Notebooks/input/human_and_dog.jpg"

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/My Drive/Colab Notebooks/resources/Mask_RCNN")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.abspath("/content/drive/My Drive/Colab Notebooks/resources/Mask_RCNN/samples/mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# COCO Class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Load a random image from the images folder
image = skimage.io.imread(input_img)

# original image
plt.figure(figsize=(12,10))
skimage.io.imshow(image)

# Run detection
results = model.detect([image], verbose=1)


# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

mask = results[0]['masks']
mask = mask.astype(int)
#mask.shape

for i, id in enumerate(r['class_ids']):
  print(i+1, class_names[id])

choice = int(input("Enter choice: "))
choice -= 1

#
#
#######################
maxwidth = 100
maxheight = 100

fig = plt.figure(figsize=(25, 25))
x, y = np.meshgrid(np.arange(maxwidth), np.arange(maxheight))

plt.plot(x, y, marker='o', color='w', markersize=10, markeredgecolor='k')

##############
img = Image.open(input_img)
img = img.convert("RGBA")
pixdata = img.load()
width, height = img.size

for y in range(height):
    for x in range(width):
      if(mask[y,x,choice] == 0):
        pixdata[x,y] = (0,0,0,0)

#img.save("test.png")

img.thumbnail((maxwidth, maxheight), Image.ANTIALIAS)
pixdata = img.load()
width, height = img.size

offsetwidth = int((maxwidth-width)/2)
offsetheight = int((maxheight-height)/2)

opacity_treshold = 150

for y in range(height):
    for x in range(width):
      if(pixdata[x,y][3] > opacity_treshold):
        plt.plot(offsetwidth+x, offsetheight+height-y, marker='o', color='k')
##############

plt.axis('off')
plt.show()
#######################
#
#

Output hidden; open in https://colab.research.google.com to view.